# Browser Automation with Playwright Integration

This notebook demonstrates integrating browser automation capabilities into LangGraph agents using the **Playwright** library. The agent can navigate websites, extract text, click elements, and interact with web pages—enabling robust web scraping and RPA (Robotic Process Automation) workflows.

**Use Cases:**
- Competitive intelligence gathering
- Automated form filling
- Dynamic content extraction
- End-to-end testing automation

In [ ]:
# Import dependencies
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
import nest_asyncio
import requests
import os
import textwrap

In [ ]:
# Initialize Environment
load_dotenv(override=True)

# Enable nested event loops (required for Jupyter + Playwright)
nest_asyncio.apply()

## Phase 1: Configure Playwright Browser Tools

LangChain provides a pre-built toolkit that exposes Playwright browser actions as agent-compatible tools.

In [ ]:
# Initialize Playwright Browser
async_browser = create_async_playwright_browser(headless=False)  # Visible browser window
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
browser_tools = toolkit.get_tools()

# Inspect Available Tools
for tool in browser_tools:
    print(f"{tool.name}: {tool.description}")

In [ ]:
# Test: Navigate and Extract Text
tool_dict = {tool.name: tool for tool in browser_tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

# Execute Navigation
await navigate_tool.arun({"url": "https://www.cnn.com"})
text = await extract_text_tool.arun({})

# Display Extracted Content
print(textwrap.fill(text[:500]))

## Phase 2: Add Notification Tool

In [ ]:
# Push Notification Tool
def push(text: str):
    """Send push notification via Pushover"""
    token = os.getenv("PUSHOVER_TOKEN")
    user = os.getenv("PUSHOVER_USER")
    if token and user:
        requests.post("https://api.pushover.net/1/messages.json",
                      data={"token": token, "user": user, "message": text})

tool_push = Tool(
    name="send_push_notification",
    func=push,
    description="Send a push notification to the user"
)

all_tools = browser_tools + [tool_push]

## Phase 3: Build Agent Graph with Browser Automation

In [ ]:
# Define State
class State(TypedDict):
    messages: Annotated[list, add_messages]

# LLM with Tools
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)

# Chatbot Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build Graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Compile with Memory
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

## Execute Browser-Enabled Agent

In [ ]:
# Interactive Agent with Browser Automation
config = {"configurable": {"thread_id": "browser_session_1"}}

async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

### Example Prompts:
- "Navigate to https://news.ycombinator.com and extract the top 3 article titles"
- "Go to weather.com and tell me the forecast for San Francisco"
- "Visit https://github.com/trending and notify me of the #1 trending repository"